In [1]:
import numpy as np
import gc
import timeit
import torch 

# Concat large arrays 

In [2]:
real_shape = (206928225, 100)
testshape = (int(206928225/100), 100)
testshape
test = True
no_tries = 3

In [3]:
if test:
    shape = testshape
else:
    shape = real_shape

In [4]:
def fast_concat(se,pe,oe):
    
    assert se.shape == pe.shape, "Error! fast_concat with differing shapes"
    assert se.shape == oe.shape, "Error! fast_concat with differing shapes"
       
    x = np.empty((se.shape[0],se.shape[1]*3),dtype=np.float32)
    x[:,0:100] =se
    x[:,100:200] = pe
    x[:,200:] = oe
    
    return x

def insert_row_into_array_np(arr,row,col_ix):
    col = np.repeat(row,len(arr)).reshape((len(arr),col_ix[1]-col_ix[0]))
    arr[:,col_ix[0]:col_ix[1]] = col
    return arr

def insert_row_into_array_zero_np(arr,row,col_ix):
    col = np.repeat(row,len(arr)).reshape((len(arr),col_ix[1]-col_ix[0]))
    arr[:,col_ix[0]:col_ix[1]] = 0
    arr[:,col_ix[0]:col_ix[1]] += col
    
    return arr

def insert_row_into_array_loop_np(arr,row,col_ix):
    for i in range(len(arr)):
        arr[:,col_ix[0]:col_ix[1]][i] = row
    return arr

def insert_row_into_array_loop_zero_np(arr,row,col_ix):
    arr[:,col_ix[0]:col_ix[1]] = 0
    for i in range(len(arr)):
        arr[:,col_ix[0]:col_ix[1]][i] += row
    return arr

In [5]:
def insert_row_into_array_torch(arr,row,col_ix):
    col = row.repeat(len(arr),1)
    arr[:,col_ix[0]:col_ix[1]] = col
    return arr

def insert_row_into_array_zero_torch(arr,row,col_ix):
    col = row.repeat(len(arr),1)
    arr[:,col_ix[0]:col_ix[1]] = 0
    arr[:,col_ix[0]:col_ix[1]] += col
    return arr

def insert_row_into_array_loop_torch(arr,row,col_ix):
    for i in range(len(arr)):
        arr[:,col_ix[0]:col_ix[1]][i] = row
    return arr

def insert_row_into_array_loop_zero_torch(arr,row,col_ix):
    arr[:,col_ix[0]:col_ix[1]] = 0
    for i in range(len(arr)):
        arr[:,col_ix[0]:col_ix[1]][i] += row
    return arr

In [6]:
testrow = np.zeros(shape[1])
testarray = np.empty((shape[0],shape[1]*3))

try_np = f"insert_row_into_array_np(testarray,testrow,(100,200));gc.collect()"
try_np_zero= f"insert_row_into_array_zero_np(testarray,testrow,(100,200));gc.collect()"
try_np_loop= f"insert_row_into_array_loop_np(testarray,testrow,(100,200));gc.collect()"
try_np_loop_zero = f"insert_row_into_array_loop_zero_np(testarray,testrow,(100,200));gc.collect()"

## Time numpy functions

In [7]:
timeit.timeit(try_np, number=no_tries,globals=globals())

4.183770211000592

In [8]:
timeit.timeit(try_np_zero, number=no_tries,globals=globals())

5.1471743549991515

In [9]:
timeit.timeit(try_np_loop, number=no_tries,globals=globals())

4.762227656001414

In [10]:
timeit.timeit(try_np_loop_zero, number=no_tries,globals=globals())

9.097452388999955

## Try torch functions

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
testrow = torch.tensor(testrow).to(device)
testarray = torch.tensor(testarray).to(device)

try_torch = f"insert_row_into_array_torch(testarray,testrow,(100,200));gc.collect()"
try_torch_zero= f"insert_row_into_array_zero_torch(testarray,testrow,(100,200));gc.collect()"
try_torch_loop= f"insert_row_into_array_loop_torch(testarray,testrow,(100,200));gc.collect()"
try_torch_loop_zero= f"insert_row_into_array_loop_zero_torch(testarray,testrow,(100,200));gc.collect()"

In [12]:
timeit.timeit(try_torch, number=no_tries,globals=globals())

1.246834547000617

In [13]:
timeit.timeit(try_torch_zero, number=no_tries,globals=globals())

2.187637431001349

In [14]:
timeit.timeit(try_torch_loop, number=no_tries,globals=globals())

29.62641365700074

In [15]:
timeit.timeit(try_torch_loop_zero, number=no_tries,globals=globals())

47.60296642600042

In [16]:
# pytorch model
import torchmetrics
import torch
from torch import nn 


class ClassifierSimple(torch.nn.Module):
    def __init__(self,input_dim=300,hidden_size=64):
        super(ClassifierSimple, self).__init__()
        
        
        self.layers = nn.Sequential(
                # flatten input if necessary
                nn.Flatten(),
                nn.Linear(input_dim,hidden_size),
                nn.ReLU(),
                nn.Linear(hidden_size,1)
        )
        
        self.output_activation = nn.Sigmoid()
                
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
 
        
    
    def forward(self,x):        
        
        return self.layers(x)
    def predict(self,x):
        x.to(self.device)
        
        return self.output_activation(self.layers(x))
    def predict_numpy(self,x):
        x = torch.tensor(x)
        x.to(self.device)
        return self.output_activation(self.layers(x)).detach().cpu().numpy()
        
    


2022-11-08 10:42:20.660479: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-08 10:42:20.660499: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Test Prediction function

In [17]:
model = ClassifierSimple()
model = model.to(device)

In [18]:
test = torch.empty((shape[0],shape[1]*3)).to(device)

In [19]:
timeit.timeit(f"model(test);gc.collect()", number=10,globals=globals())

8.043075576000774

In [20]:
timeit.timeit(f"model.predict_numpy(test);gc.collect()", number=10,globals=globals())

/tmp/ipykernel_21827/656962526.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x)


11.475482238998666

No Difference on cpu ... need to check on gpu